## 4.1 Randomized Controlled Trial
We will now deal with the randomization process, starting from a dataset by Matheus Facture, freely released on his GitHub account under the MIT license: [link](https://github.com/matheusfacure/python-causality-handbook/blob/master/causal-inference-for-the-brave-and-true)

In [3]:
# Import necessary libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [4]:
# Import wage dataset from URL
url = "https://raw.githubusercontent.com/matheusfacure/python-causality-handbook/refs/heads/master/causal-inference-for-the-brave-and-true/data/wage.csv"
wage_data = pd.read_csv(url)
wage_data = wage_data[wage_data.columns[:9]]

wage_data.head()

,wage,hours,lhwage,IQ,educ,exper,tenure,age,married
0,769,40,2.956212,93,12,11,2,31,1
1,808,50,2.782539,119,18,11,16,37,1
2,825,40,3.026504,108,14,11,9,33,1
3,650,40,2.788093,96,12,13,7,32,1
4,562,40,2.642622,74,11,14,5,34,1


In [5]:
# Initialize random seed. Number of your choice, 42 by convention. Keep the same number to ensure reproducibility
np.random.seed(42)

In [6]:
# Randomly divide data into Treatment and Control group
wage_data['group'] = np.random.choice(['Treatment', 'Control'], size=len(wage_data))
wage_data.head()

,wage,hours,lhwage,IQ,educ,exper,tenure,age,married,group
0,769,40,2.956212,93,12,11,2,31,1,Treatment
1,808,50,2.782539,119,18,11,16,37,1,Control
2,825,40,3.026504,108,14,11,9,33,1,Treatment
3,650,40,2.788093,96,12,13,7,32,1,Treatment
4,562,40,2.642622,74,11,14,5,34,1,Treatment


In [7]:
wage_data['group'].value_counts()

group
Control      473
Treatment    462
Name: count, dtype: int64

The above cell splits the code into treatment and control group **following a totally random process**, which results in a dataset that is not necessary balanced. If we want our treatment and control group to contain exactly half of our data, we can do as follows:

In [8]:
split_half = np.array(['Treatment'] * (len(wage_data) // 2) + ['Control'] * (len(wage_data) - len(wage_data) // 2))
np.random.shuffle(split_half)
wage_data['group'] = split_half

In [9]:
wage_data.head()
wage_data['group'].value_counts()

group
Control      468
Treatment    467
Name: count, dtype: int64

We can also choose to split according to other percentages, e.g. 30% and 70%:

In [10]:
wage_data['group'] = np.random.choice(['Treatment', 'Control'], size=len(wage_data), p=[0.3, 0.7])

In [11]:
wage_data.head()
wage_data['group'].value_counts()

group
Control      656
Treatment    279
Name: count, dtype: int64

Another option is **stratified randomization**: we take into account a specific variable and divide it equally between treatment and control group.

In [12]:
def stratified_randomize(subgroup):
    assignments = np.random.choice(['Treatment', 'Control'], size=len(subgroup))
    subgroup['group'] = assignments
    return subgroup

# Example: stratify by gender
df = wage_data.groupby('educ', group_keys=False).apply(stratified_randomize)
df.head()

,wage,hours,lhwage,IQ,educ,exper,tenure,age,married,group
0,769,40,2.956212,93,12,11,2,31,1,Control
1,808,50,2.782539,119,18,11,16,37,1,Treatment
2,825,40,3.026504,108,14,11,9,33,1,Control
3,650,40,2.788093,96,12,13,7,32,1,Treatment
4,562,40,2.642622,74,11,14,5,34,1,Treatment


### What can we do with RCTs?
Let's try to estimate the difference in differences:

In [13]:
effect = wage_data[wage_data['group'] == 'Treatment']['lhwage'].mean() - wage_data[wage_data['group'] == 'Control']['lhwage'].mean()
print(f"Estimated treatment effect: {effect:.2f}")

Estimated treatment effect: 0.06


In [14]:
# Simple model (only treatment)
model = smf.ols('lhwage ~ C(group)', data=wage_data).fit()
print(model.summary().tables[1])

# Add covariates (optional)
model = smf.ols('lhwage ~ C(group) + age + educ + exper', data=df).fit()


                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 2.9902      0.018    169.152      0.000       2.956       3.025
C(group)[T.Treatment]     0.0614      0.032      1.896      0.058      -0.002       0.125


In [15]:
model = smf.ols('lhwage ~ C(group) + age + educ + exper', data=df).fit()
print(model.summary().tables[1])


                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 1.5673      0.175      8.961      0.000       1.224       1.911
C(group)[T.Treatment]     0.0318      0.028      1.126      0.261      -0.024       0.087
age                       0.0100      0.005      1.842      0.066      -0.001       0.021
educ                      0.0674      0.008      8.982      0.000       0.053       0.082
exper                     0.0161      0.004      3.713      0.000       0.008       0.025
